In [30]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import seaborn as sns
from datetime import datetime
import random
import os
import optuna

In [2]:
def set_all_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

DEFAULT_RANDOM_SEED = 84
set_all_seeds(seed=DEFAULT_RANDOM_SEED)

In [3]:
path_big_train = '../train/data/train.csv'
path_big_test = '../train/data/test.csv'
df_train = pd.read_csv(path_big_train)
df_test = pd.read_csv(path_big_test)

/tmp/ipykernel_150156/3749464350.py:3: DtypeWarning: Columns (64,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(path_big_train)
/tmp/ipykernel_150156/3749464350.py:4: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv(path_big_test)


In [4]:
import re

targets = ['target_month', 'target_day']
numerical = list(filter(lambda x: re.match(r'\d+[dm]_', x), df_train.columns))
categorical = ['kod_vrab', 'model',  'zavod_build', 'kuzov', 'telega', 'expected_srok_sl_y_b', 'date_build_b', 'manage_type', 'rod_id_x', 'reestr_state',
                'tormoz', 'tipvozd', 'tippogl', 'ownertype']
dates = ['month', 'date_kap', 'date_dep', 'date_pl_rem', 'last_rem_date', 'date_build', 'srok_sl']

numerical += [
    "days_to_pl_rem",
    "days_from_last_rem",
    "ost_prob",
    "gruz",
    "cnsi_gruz_capacity",
    "cnsi_volumek",
    "tara",
    "cnsi_probeg_dr",
    "cnsi_probeg_kr",
    "norma_km",
    "expected_srok_sl_y"
]

numerical += ['kod_vrab_2_tr',
              'kod_vrab_3_tr', 'kod_vrab_5_tr', 'neis1_kod_0_tr', 'neis1_kod_2_tr',
              'neis1_kod_95_tr', 'neis1_kod_6_tr', 'neis1_kod_88_tr',
              'neis1_kod_tr_max', 'neis2_kod_98_tr', 'neis2_kod_95_tr',
              'neis2_kod_96_tr', 'neis2_kod_62_tr', 'neis2_kod_tr_max',
              'neis3_kod_54_tr', 'neis3_kod_51_tr', 'neis3_kod_tr_max',
              'mod1_kod_tr_min', 'mod1_kod_tr_max', 'mod2_kod_tr_min',
              'mod2_kod_tr_max', 'gr_probeg_tr_max', 'gr_probeg_tr_mean',
              'gr_probeg_tr_min', 'por_probeg_tr_max', 'por_probeg_tr_mean',
              'por_probeg_tr_min', '_1m_tr_rems', 'kod_vrab_0_pr', 'kod_vrab_1_pr',
              'diff_road_pr', 'diff_st_pr', '_10d_pr_rems', '_20d_pr_rems',
              '_1m_pr_rems', 'rem_dist_pr_sum', 'rem_dist_pr_mean', '_allm_pr_rems',
              '_allm_tr_rems']

notna_features = ['date_kap', 'date_dep', 'date_iskl']
notna_features_res = [feature + '_notna' for feature in notna_features]
categorical += notna_features_res

print('Unused columns:')
for col in df_train.columns:
    if col not in targets and col not in numerical and col not in categorical and col not in dates:
        print(col)

features = numerical + categorical + targets
train_features = numerical + categorical

medians = df_train[numerical].median()

def preprcoess_df(df, test=False):
    for date in dates:
        df[date] = pd.to_datetime(df[date])
    df = df[df['month'] > '2022-08-01']

    for col in notna_features:
        df[col + '_notna'] = df[col].notna().astype(int)

    features = numerical + categorical + targets
    if test:
        features = train_features
    df = df[features]

    df[categorical] = df[categorical].astype('str')
    df[numerical] = df[numerical].fillna(medians)

    return df

df_train = preprcoess_df(df_train)
df_test = preprcoess_df(df_test, test=True)

Unused columns:
wagnum
ownership_type
rod_id_y
date_iskl


/tmp/ipykernel_150156/1696388158.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + '_notna'] = df[col].notna().astype(int)
/tmp/ipykernel_150156/1696388158.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col + '_notna'] = df[col].notna().astype(int)
/tmp/ipykernel_150156/1696388158.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

In [5]:
df_train.shape, df_test.shape

((203586, 129), (33707, 127))

In [6]:
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold

def train(target):
    cnt_folds = 3
    clfs = []
    scores = []
    kf = StratifiedKFold(n_splits=cnt_folds, shuffle=True, random_state=DEFAULT_RANDOM_SEED)

    features = numerical + categorical + [target]
    df = df_train[features]
    X = df.drop(target, axis=1)
    y = df[target]

    for ind, (train_index, test_index) in enumerate(kf.split(X=X, y=y)):

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        clf = CatBoostClassifier(cat_features=categorical,
                                 iterations=400,
                                 eval_metric='F1',
                                 random_seed=DEFAULT_RANDOM_SEED,
                                 allow_writing_files=False)

        clf.fit(X_train, y_train,
                eval_set=(X_test, y_test),
                verbose=150, plot=False)

        clfs.append(clf)
        y_pred = clf.predict(X_test)
        score = f1_score(y_test, y_pred)
        scores.append(score)
        print(f'{ind} model: {score} f1')

    assert len(clfs) == cnt_folds

    print("mean f1 score --", np.mean(scores, dtype="float16"), np.std(scores).round(4))

    return clfs, np.mean(scores, dtype="float16"), np.std(scores).round(4)

In [10]:
month_clfs, month_f1, month_std = train('target_month')
day_clf, day_f1, day_std = train('target_day')

Learning rate set to 0.158774
0:	learn: 0.5451649	test: 0.5392612	best: 0.5392612 (0)	total: 291ms	remaining: 1m 56s
150:	learn: 0.7233408	test: 0.6727059	best: 0.6770519 (78)	total: 24.4s	remaining: 40.2s
300:	learn: 0.7744758	test: 0.6677852	best: 0.6770519 (78)	total: 45.1s	remaining: 14.8s
399:	learn: 0.7987820	test: 0.6672252	best: 0.6770519 (78)	total: 1m 1s	remaining: 0us

bestTest = 0.6770519263
bestIteration = 78

Shrink model to first 79 iterations.
0 model: 0.6770519262981575 f1
Learning rate set to 0.158774
0:	learn: 0.4396607	test: 0.4374199	best: 0.4374199 (0)	total: 178ms	remaining: 1m 11s
150:	learn: 0.7129195	test: 0.6799670	best: 0.6817208 (144)	total: 29s	remaining: 47.9s
300:	learn: 0.7587814	test: 0.6815716	best: 0.6834817 (228)	total: 49.6s	remaining: 16.3s
399:	learn: 0.7861989	test: 0.6744800	best: 0.6834817 (228)	total: 1m 2s	remaining: 0us

bestTest = 0.6834817013
bestIteration = 228

Shrink model to first 229 iterations.
1 model: 0.6834817012858555 f1
Learnin

In [12]:
print(month_f1, month_std)
print(day_f1, day_std)

0.6772 0.0054
0.5444 0.0086


In [13]:
(month_f1 + day_f1) / 2

0.61083984375

In [14]:
cnt_folds = 3
df_features = sum(month_clfs[i].get_feature_importance(prettified=True).set_index('Feature Id') for i in range(cnt_folds)) / cnt_folds
df_features = df_features.sort_values('Importances', ascending=False)

In [28]:
df_features

,Importances
Feature Id,
ost_prob,14.070812
days_to_pl_rem,11.467984
5d_ost_prob_max,11.214305
5d_ost_prob_min,9.660306
10d_ost_prob_max,7.569389
...,...
diff_road_pr,0.000002
10d_skoroport_mean,0.000000
5d_skoroport_mean,0.000000


In [35]:
def opt_train(trial):
    target = 'target_month'

    param = {
        'iterations': trial.suggest_int('iterations', 100, 500),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'random_strength': trial.suggest_int('random_strength', 0, 100),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide'])
    }

    clf = CatBoostClassifier(cat_features=categorical,
                             eval_metric='F1',
                             random_seed=DEFAULT_RANDOM_SEED,
                             allow_writing_files=False,
                             **param)

    features = numerical + categorical + [target]
    df = df_train[features]
    X = df.drop(target, axis=1)
    y = df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=DEFAULT_RANDOM_SEED)

    clf.fit(X_train, y_train,
            eval_set=(X_test, y_test),
            verbose=150, plot=False)

    y_pred = clf.predict(X_test)
    score = f1_score(y_test, y_pred)

    return score

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(opt_train, n_trials=100) # Можно изменить количество итераций

print("Лучшие параметры: ", study.best_params)

[I 2023-11-12 04:44:12,303] A new study created in memory with name: no-name-f68465eb-29f4-41c7-b256-ec96c2c15cc2


0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 231ms	remaining: 34.4s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.666553768
bestIteration = 82

Shrink model to first 83 iterations.


[I 2023-11-12 04:44:35,854] Trial 0 finished with value: 0.666553767993226 and parameters: {'iterations': 150, 'depth': 8, 'learning_rate': 0.25764691338078466, 'random_strength': 83, 'bagging_temperature': 0.39125301527748746, 'od_type': 'Iter', 'grow_policy': 'SymmetricTree'}. Best is trial 0 with value: 0.666553767993226.


0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 105ms	remaining: 46.9s
150:	learn: 0.8324799	test: 0.6643461	best: 0.6685515 (96)	total: 16.5s	remaining: 32.4s
300:	learn: 0.9249509	test: 0.6589557	best: 0.6685515 (96)	total: 30.3s	remaining: 14.8s
447:	learn: 0.9688359	test: 0.6595148	best: 0.6685515 (96)	total: 53.9s	remaining: 0us

bestTest = 0.6685514718
bestIteration = 96

Shrink model to first 97 iterations.


[I 2023-11-12 04:45:31,975] Trial 1 finished with value: 0.668551471811076 and parameters: {'iterations': 448, 'depth': 9, 'learning_rate': 0.29990282741361146, 'random_strength': 43, 'bagging_temperature': 0.1946616268414929, 'od_type': 'IncToDec', 'grow_policy': 'Lossguide'}. Best is trial 1 with value: 0.668551471811076.


...

0:	learn: 0.2409085	test: 0.2414467	best: 0.2414467 (0)	total: 169ms	remaining: 1m 8s
150:	learn: 0.8000990	test: 0.6683426	best: 0.6704679 (132)	total: 20.6s	remaining: 34.9s
300:	learn: 0.9571318	test: 0.6712536	best: 0.6750335 (276)	total: 43.8s	remaining: 15.4s
406:	learn: 0.9871419	test: 0.6689121	best: 0.6750335 (276)	total: 59.5s	remaining: 0us

bestTest = 0.6750334672
bestIteration = 276

Shrink model to first 277 iterations.


[I 2023-11-12 04:56:10,681] Trial 23 finished with value: 0.675033467202142 and parameters: {'iterations': 407, 'depth': 9, 'learning_rate': 0.12193882879165052, 'random_strength': 12, 'bagging_temperature': 0.9942026304071137, 'od_type': 'IncToDec', 'grow_policy': 'Depthwise'}. Best is trial 23 with value: 0.675033467202142.


0:	learn: 0.2896936	test: 0.3017724	best: 0.3017724 (0)	total: 197ms	remaining: 1m 18s
150:	learn: 0.8481983	test: 0.6676712	best: 0.6715621 (137)	total: 28.7s	remaining: 47s
300:	learn: 0.9839500	test: 0.6674508	best: 0.6715621 (137)	total: 1m	remaining: 19.5s
397:	learn: 0.9964928	test: 0.6661060	best: 0.6715621 (137)	total: 1m 16s	remaining: 0us

bestTest = 0.6715620828
bestIteration = 137

Shrink model to first 138 iterations.


[I 2023-11-12 04:57:29,283] Trial 24 finished with value: 0.6715620827770361 and parameters: {'iterations': 398, 'depth': 10, 'learning_rate': 0.112379946021708, 'random_strength': 11, 'bagging_temperature': 0.9638715178897177, 'od_type': 'IncToDec', 'grow_policy': 'Depthwise'}. Best is trial 23 with value: 0.675033467202142.


0:	learn: 0.2269781	test: 0.2213400	best: 0.2213400 (0)	total: 160ms	remaining: 54.9s
